In [16]:
!mkdir input_video
!mkdir inputImages
!mkdir outputImages
!mkdir output_video

mkdir: cannot create directory ‘input_video’: File exists
mkdir: cannot create directory ‘inputImages’: File exists
mkdir: cannot create directory ‘outputImages’: File exists
mkdir: cannot create directory ‘output_video’: File exists


In [17]:
#python code to clean the  folders!! do not run before analysis
import os
import shutil

def clean_folder(folder_path):
    try:
        # Check if the folder exists
        if os.path.exists(folder_path):
            # Iterate over all the files and subfolders in the directory
            for filename in os.listdir(folder_path):
                file_path = os.path.join(folder_path, filename)

                # Check if it's a file or directory
                if os.path.isfile(file_path):
                    os.remove(file_path)
                    print(f"Deleted file: {file_path}")
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
                    print(f"Deleted folder: {file_path}")
            print("Folder cleaned successfully.")
        else:
            print("Folder does not exist.")
    except Exception as e:
        print(f"Error: {e}")

# Example usage

In [18]:
folder_path1 = "/content/inputImages"
folder_path2 = "/content/outputImages"
folder_path3 = "/content/input_video"
folder_path4 = "/content/output_video"
clean_folder(folder_path1)
clean_folder(folder_path2)
clean_folder(folder_path3)
clean_folder(folder_path4)

Folder cleaned successfully.
Folder cleaned successfully.
Folder cleaned successfully.
Folder cleaned successfully.


In [19]:
xmax=240

In [20]:
!pip install ultralytics

In [21]:
import glob
import os
import cv2

# Specify the directory to search for video files
video_directory = 'input_video'

# Define the pattern to search for video files (e.g., .mp4, .avi)
video_patterns = ['*.mp4', '*.avi', '*.mkv']  # Add other patterns if needed

# Initialize a list to store found video files
video_files = []

# Search for video files using the defined patterns
for pattern in video_patterns:
    video_files.extend(glob.glob(os.path.join(video_directory, pattern)))

# Check if any video files were found
if video_files:
    print(f"Found video files: {video_files}")
    # Use the first video file found, or choose one based on your logic
    video_path = video_files[0]
    print(f"Using video file: {video_path}")
else:
    print("No video files found in the specified directory.")

Found video files: ['input_video/animated2.mp4']
Using video file: input_video/animated2.mp4


In [22]:
video_path

# Specify the directory where frames will be saved
output_dir = 'inputImages'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the original frame rate of the video
original_fps = cap.get(cv2.CAP_PROP_FPS)
print(original_fps)

30.0


In [23]:
desired_fps = 20  # Adjust this to your desired frame rate


frame_interval = int(original_fps / desired_fps)

frame_count = 0
extracted_frame_count = 0

while True:
    # Read the next frame from the video
    ret, frame = cap.read()

    # If the frame was not successfully read, we've reached the end of the video
    if not ret:
        break

    # Extract the frame if it's at the right interval
    if frame_count % frame_interval == 0:
        frame_filename = os.path.join(output_dir, f'frame_{extracted_frame_count:04d}.jpg')
        cv2.imwrite(frame_filename, frame)
        print(f'Saved {frame_filename}')
        extracted_frame_count += 1

    frame_count += 1

# Release the video capture object
cap.release()

Saved inputImages/frame_0000.jpg
Saved inputImages/frame_0001.jpg
Saved inputImages/frame_0002.jpg
Saved inputImages/frame_0003.jpg
Saved inputImages/frame_0004.jpg
Saved inputImages/frame_0005.jpg
Saved inputImages/frame_0006.jpg
Saved inputImages/frame_0007.jpg
Saved inputImages/frame_0008.jpg
Saved inputImages/frame_0009.jpg
Saved inputImages/frame_0010.jpg
Saved inputImages/frame_0011.jpg
Saved inputImages/frame_0012.jpg
Saved inputImages/frame_0013.jpg
Saved inputImages/frame_0014.jpg
Saved inputImages/frame_0015.jpg
Saved inputImages/frame_0016.jpg
Saved inputImages/frame_0017.jpg
Saved inputImages/frame_0018.jpg
Saved inputImages/frame_0019.jpg
Saved inputImages/frame_0020.jpg
Saved inputImages/frame_0021.jpg
Saved inputImages/frame_0022.jpg
Saved inputImages/frame_0023.jpg
Saved inputImages/frame_0024.jpg
Saved inputImages/frame_0025.jpg
Saved inputImages/frame_0026.jpg
Saved inputImages/frame_0027.jpg
Saved inputImages/frame_0028.jpg
Saved inputImages/frame_0029.jpg
Saved inpu

In [24]:
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.2.89 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.9/112.6 GB disk)


In [25]:
# Load the model
model = YOLO('best.pt')
results = model.predict(source='inputImages', conf=0.25, save=True)

output_dir = "outputImages"
os.makedirs(output_dir, exist_ok=True)

# Loop through each result to access predictions
for result in results:
    # Predicted image path
    image_path = result.path  # Path to the predicted image file
    image = cv2.imread(image_path)  # Read the image using OpenCV

    # Labels and bounding boxes
    labels = result.names  # Class labels dictionary (e.g., {0: 'football'})
    boxes = result.boxes  # List of bounding boxes and confidences

    print(f'Predicted Image: {image_path}')
    if len(boxes) == 0:
      output_path = os.path.join(output_dir, os.path.basename(image_path))
      cv2.line(image, (xmax,0), (xmax, 640), color=(0,0,0), thickness=2)
      cv2.imwrite(output_path, image)
      continue

    maxbox=boxes[0]
    for box in boxes:
        cls_id = int(box.cls[0])  # Class ID
        label = labels[cls_id]    # Corresponding label
        confidence = box.conf[0]  # Confidence score
        bbox = box.xyxy[0]        # Bounding box in xyxy format
        print(f'Label: {label}, Confidence: {confidence:.2f}, BBox: {bbox}')

        output_path = os.path.join(output_dir, os.path.basename(image_path))
        if(maxbox.conf[0]>box.conf[0]):
          maxbox=box
    if(maxbox.xyxy[0][2]-xmax<=0):
      cv2.line(image, (xmax,0), (xmax, 640), color=(124,34,106), thickness=2)
      cv2.rectangle(
    image,
    (int(maxbox.xyxy[0][0]), int(maxbox.xyxy[0][1])),   # Top-left corner (x_min, y_min)
    (int(maxbox.xyxy[0][2]), int(maxbox.xyxy[0][3])),   # Bottom-right corner (x_max, y_max)
    color=(0, 255, 0),
    thickness=3
)

      output_path = os.path.join(output_dir, os.path.basename(image_path))
      cv2.imwrite(output_path, image)
    else:
      cv2.line(image, (xmax,0), (xmax, 640), color=(0,0,0), thickness=2)
      cv2.rectangle(
    image,
    (int(maxbox.xyxy[0][0]), int(maxbox.xyxy[0][1])),   # Top-left corner (x_min, y_min)
    (int(maxbox.xyxy[0][2]), int(maxbox.xyxy[0][3])),   # Bottom-right corner (x_max, y_max)
    color=(255, 0, 0),
    thickness=3
)

      output_path = os.path.join(output_dir, os.path.basename(image_path))
      cv2.imwrite(output_path, image)




image 1/234 /content/inputImages/frame_0000.jpg: 384x640 1 football, 11.4ms
image 2/234 /content/inputImages/frame_0001.jpg: 384x640 1 football, 10.7ms
image 3/234 /content/inputImages/frame_0002.jpg: 384x640 (no detections), 10.6ms
image 4/234 /content/inputImages/frame_0003.jpg: 384x640 (no detections), 10.6ms
image 5/234 /content/inputImages/frame_0004.jpg: 384x640 1 football, 10.6ms
image 6/234 /content/inputImages/frame_0005.jpg: 384x640 1 football, 10.7ms
image 7/234 /content/inputImages/frame_0006.jpg: 384x640 1 football, 10.6ms
image 8/234 /content/inputImages/frame_0007.jpg: 384x640 1 football, 10.6ms
image 9/234 /content/inputImages/frame_0008.jpg: 384x640 1 football, 11.5ms
image 10/234 /content/inputImages/frame_0009.jpg: 384x640 1 football, 10.7ms
image 11/234 /content/inputImages/frame_0010.jpg: 384x640 1 football, 11.6ms
image 12/234 /content/inputImages/frame_0011.jpg: 384x640 1 football, 10.9ms
image 13/234 /content/inputImages/frame_0012.jpg: 384x640 1 football, 10.5

In [26]:
import os
import glob
import cv2

frames_dir = 'outputImages'
output_video_path = 'output_video/animated_video.mp4'
frame_files = sorted(glob.glob(os.path.join(frames_dir, '*.jpg')))


if not frame_files:
    raise ValueError("No frames found in the specified directory.")


first_frame = cv2.imread(frame_files[0])
height, width, layers = first_frame.shape


fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can also use 'XVID', 'MJPG', etc.
# fps = 10  # Frames per second for the output video
# fps = desired_fps/2 #to make slow
fps = 5

# Create the VideoWriter object
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Iterate through all the frames and write them to the video
for frame_file in frame_files:
    frame = cv2.imread(frame_file)
    out.write(frame)

# Release the VideoWriter object
out.release()

print(f"Video saved as {output_video_path}")

Video saved as output_video/animated_video.mp4
